# 1

In [0]:
import torch 
import torch.nn as nn 
import torchvision 
import torchvision.transforms as transforms

In [0]:
sequence_length = 28 
input_size = 28 
hidden_size = 128 
num_layers = 2 
num_classes = 10 
batch_size = 100 
num_epochs = 2 
learning_rate = 0.01 

In [0]:
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

0it [00:00, ?it/s]

100%|█████████▉| 9912320/9912422 [00:27<00:00, 232481.04it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



0it [00:00, ?it/s]


 57%|█████▋    | 16384/28881 [00:00<00:00, 86280.38it/s]
32768it [00:00, 57252.08it/s]                           
0it [00:00, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



 96%|█████████▌| 1581056/1648877 [00:04<00:00, 528217.19it/s]

0it [00:00, ?it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw




  0%|          | 0/4542 [00:00<?, ?it/s]

8192it [00:00, 20583.77it/s]            

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw
Processing...
Done!


In [0]:
class RNN(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers, num_classes):
    super(RNN, self).__init__()
    self.hidden_size = hidden_size 
    self.num_layers = num_layers 
    self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
    self.fc = nn.Linear(hidden_size, num_classes)

  def forward(self, x):
    h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
    c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)

    out, _ = self.lstm(x, (h0, c0))

    out = self.fc(out[:, -1, :])
    return out 

In [0]:
model = RNN(input_size, hidden_size, num_layers, num_classes)

In [0]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [0]:
len(train_loader)

600

In [0]:
total_step = len(train_loader)
for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(train_loader):
    images = images.reshape(-1, sequence_length, input_size)
    labels = labels

    outputs = model(images)
    loss = criterion(outputs, labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (i+1) % 100 == 0:
      print('Epoch [{}/{}], Step [{}/{}], Loss:{:.4f}'.format(epoch+1,num_epochs, i+1, total_step, loss.item()))

Epoch [1/2], Step [100/600], Loss:0.6740
Epoch [1/2], Step [200/600], Loss:0.3524
Epoch [1/2], Step [300/600], Loss:0.2426
Epoch [1/2], Step [400/600], Loss:0.1210
Epoch [1/2], Step [500/600], Loss:0.2524
Epoch [1/2], Step [600/600], Loss:0.1792
Epoch [2/2], Step [100/600], Loss:0.1532
Epoch [2/2], Step [200/600], Loss:0.1403
Epoch [2/2], Step [300/600], Loss:0.0328
Epoch [2/2], Step [400/600], Loss:0.0383
Epoch [2/2], Step [500/600], Loss:0.0148
Epoch [2/2], Step [600/600], Loss:0.0415


In [0]:
with torch.no_grad():
  correct = 0 
  total = 0 
  for images, labels in test_loader:
    images

# 2

In [0]:
import torch 
import torch.nn as nn 
import torchvision 
import torchvision.transforms as transforms 

In [0]:
sequence_length = 28 
input_size = 28 
hidden_size = 128 
num_layers = 2 
num_classes = 10 
batch_size = 100 
num_epochs = 2 
learning_rate = 0.001 

In [0]:
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

  0%|          | 0/9912422 [00:00<?, ?it/s]

9920512it [00:00, 19181872.15it/s]                            


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


32768it [00:00, 311895.54it/s]                           
0it [00:00, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw


1654784it [00:00, 4740675.91it/s]                           
8192it [00:00, 110768.90it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw
Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw
Processing...
Done!


In [0]:
class BiRNN(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers, num_classes):
    super(BiRNN, self).__init__()
    self.hidden_size = hidden_size 
    self.num_layers = num_layers 
    self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
    self.fc = nn.Linear(hidden_size*2, num_classes)

  def forward(self, x):
    h0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size)
    c0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size)

    out, _ = self.lstm(x, (h0, c0))

    out = self.fc(out[:, -1, :])
    return out 

In [0]:
model = BiRNN(input_size, hidden_size, num_layers, num_classes)

In [0]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [0]:
total_step = len(train_loader)
for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(train_loader):
    images = images.reshape(-1, sequence_length, input_size)
    labels = labels 

    outputs = model(images)
    loss = criterion(outputs, labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (i+1) % 100 == 0:
      print('Epoch [{}/{}], Step [{}/{}], Loss:{:.4f}'.format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/2], Step [100/600], Loss:0.0598
Epoch [1/2], Step [200/600], Loss:0.0603
Epoch [1/2], Step [300/600], Loss:0.1302
Epoch [1/2], Step [400/600], Loss:0.1141
Epoch [1/2], Step [500/600], Loss:0.1156
Epoch [1/2], Step [600/600], Loss:0.0670
Epoch [2/2], Step [100/600], Loss:0.0266
Epoch [2/2], Step [200/600], Loss:0.0247
Epoch [2/2], Step [300/600], Loss:0.0696
Epoch [2/2], Step [400/600], Loss:0.0539
Epoch [2/2], Step [500/600], Loss:0.0565
Epoch [2/2], Step [600/600], Loss:0.0267


In [0]:
with torch.no_grad():
  correct = 0 
  total = 0 
  for images, labels in test_loader:
    images = images.reshape(-1, sequence_length, input_size)
    labels = labels 
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

  print('Accuracy: {} %'.format(100 * correct / total))

Accuracy: 96.44 %


In [0]:
torch.save(model.state_dict(), 'model.ckpt')

# 3

In [0]:
!git clone https://github.com/yunjey/pytorch-tutorial.git

Cloning into 'pytorch-tutorial'...
remote: Enumerating objects: 816, done.
remote: Total 816 (delta 0), reused 0 (delta 0), pack-reused 816
Receiving objects: 100% (816/816), 12.78 MiB | 32.16 MiB/s, done.
Resolving deltas: 100% (432/432), done.


In [0]:
!ls pytorch-tutorial/tutorials/02-intermediate/language_model/data_utils.py

LICENSE  logo  README.md  tutorials


In [0]:
!cp ./pytorch-tutorial/tutorials/02-intermediate/language_model/data_utils.py .

In [0]:
!ls

data  data_utils.py  model.ckpt  pytorch-tutorial  sample_data


In [0]:
import torch 
import torch.nn as nn 
import numpy as np 
from torch.nn.utils import clip_grad_norm_ 
from data_utils import Dictionary, Corpus

In [0]:
embed_size = 128 
hidden_size = 1024 
num_layers = 1 
num_epochs = 5 
num_samples = 1000 
batch_size = 20 
seq_length = 30 
learning_rate = 0.002 

In [0]:
corpus = Corpus()
ids = corpus.get_data('./pytorch-tutorial/tutorials/02-intermediate/language_model/data/train.txt', batch_size)
vocab_size = len(corpus.dictionary)
num_batches = ids.size(1) // seq_length

In [0]:
class RNNLM(nn.Module):
  def __init__(self, vocab_size, embed_size, hidden_size, num_layers):
    super(RNNLM, self).__init__()
    self.embed = nn.Embedding(vocab_size, embed_size)
    self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True)
    self.linear = nn.Linear(hidden_size, vocab_size)

  def forward(self, x, h):
    x = self.embed(x)

    out, (h, c) = self.lstm(x, h)

    out = out.reshape(out.size(0)*out.size(1), out.size(2))

    out = self.linear(out)
    return out, (h, c)


In [0]:
model = RNNLM(vocab_size, embed_size, hidden_size, num_layers)

In [0]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [0]:
def detach(states):
  return [state.detach() for state in states]

In [0]:
for epoch in range(num_epochs):
  states = (torch.zeros(num_layers, batch_size, hidden_size), 
        torch.zeros(num_layers, batch_size, hidden_size))

  for i in range(0, ids.size(1) - seq_length, seq_length):
    inputs = ids[:, i:i+seq_length]
    targets = ids[:, (i+1):(i+1)+seq_length]
    states = detach(states)
    outputs, states = model(inputs, states)
    loss = criterion(outputs, targets.reshape(-1))

    model.zero_grad()
    loss.backward()
    clip_grad_norm_(model.parameters(), 0.5)
    optimizer.step()

    step = (i+1) // seq_length
    if step % 100 == 0:
      print('Epoch [{}/{}], Step [{}/{}], Loss:{:.4f}, Perplexity:{:5.2f}'
          .format(epoch+1, num_epochs, step, num_batches, loss.item(), np.exp(loss.item())))

Epoch [1/5], Step [0/1549], Loss:9.2134, Perplexity:10030.72
Epoch [1/5], Step [100/1549], Loss:5.9998, Perplexity:403.34
Epoch [1/5], Step [200/1549], Loss:5.9472, Perplexity:382.68
Epoch [1/5], Step [300/1549], Loss:5.7448, Perplexity:312.55
Epoch [1/5], Step [400/1549], Loss:5.6560, Perplexity:286.01
Epoch [1/5], Step [500/1549], Loss:5.1442, Perplexity:171.44
Epoch [1/5], Step [600/1549], Loss:5.2042, Perplexity:182.03
Epoch [1/5], Step [700/1549], Loss:5.3576, Perplexity:212.23
Epoch [1/5], Step [800/1549], Loss:5.1896, Perplexity:179.39
Epoch [1/5], Step [900/1549], Loss:5.1253, Perplexity:168.23
Epoch [1/5], Step [1000/1549], Loss:5.1234, Perplexity:167.91
Epoch [1/5], Step [1100/1549], Loss:5.3373, Perplexity:207.94
Epoch [1/5], Step [1200/1549], Loss:5.1858, Perplexity:178.72
Epoch [1/5], Step [1300/1549], Loss:5.0858, Perplexity:161.70
Epoch [1/5], Step [1400/1549], Loss:4.8435, Perplexity:126.91
Epoch [1/5], Step [1500/1549], Loss:5.1224, Perplexity:167.74
Epoch [2/5], Step 

In [0]:
with torch.no_grad():
  with open('samples.txt', 'w') as f:
    state = (torch.zeros(num_layers, 1, hidden_size), 
             torch.zeros(num_layers, 1, hidden_size))

    prob = torch.ones(vocab_size)
    input = torch.multinomial(prob, num_samples=1).unsqueeze(1)
    
    for i in range(num_samples):
      output, state = model(input, state)

      prob = output.exp()
      word_id = torch.multinomial(prob, num_samples=1).item()

      input.fill_(word_id)

      word = corpus.dictionary.idx2word[word_id]
      word = '\n' if word == '<eos>' else word + ' '

      f.write(word)

      if (i+1) % 100 == 0:
        print('Sampled [{}/{}] words and save to {}'.format(i+1, num_samples, 'sample.txt'))

Sampled [100/1000] words and save to sample.txt
Sampled [200/1000] words and save to sample.txt
Sampled [300/1000] words and save to sample.txt
Sampled [400/1000] words and save to sample.txt
Sampled [500/1000] words and save to sample.txt
Sampled [600/1000] words and save to sample.txt
Sampled [700/1000] words and save to sample.txt
Sampled [800/1000] words and save to sample.txt
Sampled [900/1000] words and save to sample.txt
Sampled [1000/1000] words and save to sample.txt


In [0]:
!ls

data	       model.ckpt   pytorch-tutorial  samples.txt
data_utils.py  __pycache__  sample_data


In [0]:
torch.save(model.state_dict(), 'model1.ckpt')

In [0]:
!ls

data	       model1.ckpt  __pycache__       sample_data
data_utils.py  model.ckpt   pytorch-tutorial  samples.txt


# 4

In [0]:
import os 
import torch 
import torchvision 
import torch.nn as nn 
from torchvision import transforms 
from torchvision.utils import save_image 

In [0]:
latent_size = 64 
hidden_size = 256 
image_size = 784 
bun_epochs = 100 
batch_size = 100 
sample_dir = 'samples'

In [0]:
if not os.path.exists(sample_dir):
  os.makedirs(sample_dir)

In [0]:
torch.zeros(2, 3, 4)

tensor([[[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]]])

In [0]:
transform = transforms.Compose([
          transforms.ToTensor(), 
          transforms.Normalize(mean=(0.5, 0.5, 0.5), 
                    std=(0.5, 0.5, 0.5))
])

In [0]:
mnist = torchvision.datasets.MNIST(root='./data', train=True, transform=transform, download=True)

data_loader = torch.utils.data.DataLoader(dataset=mnist, batch_size=batch_size, shuffle=True)

In [0]:
D = nn.Sequential(
    nn.Linear(image_size, hidden_size), 
    nn.LeakyReLU(0.2), 
    nn.Linear(hidden_size, hidden_size), 
    nn.LeakyReLU(0.2), 
    nn.Linear(hidden_size, 1), 
    nn.Sigmoid()
)

G = nn.Sequential(
    nn.Linear(latent_size, hidden_size), 
    nn.ReLU(), 
    nn.Linear(hidden_size, hidden_size), 
    nn.ReLU(), 
    nn.Linear(hidden_size, image_size), 
    nn.Tanh()
)

In [0]:
criterion = nn.BCELoss()
d_optimizer = torch.optim.Adam(D.parameters(), lr=0.0002)
g_optimizer = torch.optim.Adam(G.parameters(), lr=0.0002)

In [0]:
def denorm(x):
    out = (x + 1) / 2 
    return out.clamp(0, 1)

In [0]:
def reset_grad():
    d_optimizer.zero_grad()
    g_optimizer.zero_grad()

In [0]:
total_step = len(data_loader)
# for epoch in range(num_epochs):
#     for i, (images, _) in enumerate(data_loader):
#         images = images.reshape(batch_size, -1)

#         real_labels = torch.ones(batch_size, 1)
#         fake_labels = torch.zeros(batch_size, 1)

#         outputs = D(images)
#         d_loss_real = criterion(outputs, real_labels)
#         real_score = outputs 

#         z = torch.randn(batch_size, latent_size)
#         fake_images = G(z)
#         outputs = D(fake_images)
#         d_loss_fake = criterion(outputs, fake_labels)
#         fake_score = outputs 

#         d_loss = d_loss_real + d_loss_fake 
#         reset_grad()
#         d_loss.backward()
#         d_optimizer.step()


#         # train generator 

#         z = torch.randn(batch_size, latent_size)
#         fake_images = G(z)
#         outputs = D(fake_images)

#         g_loss = criterion(outputs, real_labels)

#         reset_grad()
#         g_loss.backward()
#         g_optimizer.step()

#         if (i+1) % 200 == 0:
#             print('Epoch [{}/{}], Step [{}/{}], d_loss: {:.4f}, D(x): {:.2f}, D(G(z)): {:.2f}'
#                 .format(epoch, num_epochs, i+1, total_step, d_loss.item(), g_loss.item(), 
#                     real_score.mean().item(), fake_score.mean().item()))

#     if (epoch+1) == 1:
#         images = images.reshape(images.size(0), 1, 28, 28)
#         save_image(denorm(images), os.path.join(sample_dir, 'real_images.png'))

#     fake_images = fake_images.reshape(fake_images.size(0), 1, 28, 28)
#     save_image(denorm(fake_images), os.path.join(sample_dir, 'fake_images-{}.png'.format(epoch+1)))
for epoch in range(num_epochs):
    for i, (images, _) in enumerate(data_loader):
        images = images.reshape(batch_size, -1).to(device)
        
        # Create the labels which are later used as input for the BCE loss
        real_labels = torch.ones(batch_size, 1).to(device)
        fake_labels = torch.zeros(batch_size, 1).to(device)

        # ================================================================== #
        #                      Train the discriminator                       #
        # ================================================================== #

        # Compute BCE_Loss using real images where BCE_Loss(x, y): - y * log(D(x)) - (1-y) * log(1 - D(x))
        # Second term of the loss is always zero since real_labels == 1
        outputs = D(images)
        d_loss_real = criterion(outputs, real_labels)
        real_score = outputs
        
        # Compute BCELoss using fake images
        # First term of the loss is always zero since fake_labels == 0
        z = torch.randn(batch_size, latent_size).to(device)
        fake_images = G(z)
        outputs = D(fake_images)
        d_loss_fake = criterion(outputs, fake_labels)
        fake_score = outputs
        
        # Backprop and optimize
        d_loss = d_loss_real + d_loss_fake
        reset_grad()
        d_loss.backward()
        d_optimizer.step()
        
        # ================================================================== #
        #                        Train the generator                         #
        # ================================================================== #

        # Compute loss with fake images
        z = torch.randn(batch_size, latent_size).to(device)
        fake_images = G(z)
        outputs = D(fake_images)
        
        # We train G to maximize log(D(G(z)) instead of minimizing log(1-D(G(z)))
        # For the reason, see the last paragraph of section 3. https://arxiv.org/pdf/1406.2661.pdf
        g_loss = criterion(outputs, real_labels)
        
        # Backprop and optimize
        reset_grad()
        g_loss.backward()
        g_optimizer.step()
        
        if (i+1) % 200 == 0:
            print('Epoch [{}/{}], Step [{}/{}], d_loss: {:.4f}, g_loss: {:.4f}, D(x): {:.2f}, D(G(z)): {:.2f}' 
                  .format(epoch, num_epochs, i+1, total_step, d_loss.item(), g_loss.item(), 
                          real_score.mean().item(), fake_score.mean().item()))
    
    # Save real images
    if (epoch+1) == 1:
        images = images.reshape(images.size(0), 1, 28, 28)
        save_image(denorm(images), os.path.join(sample_dir, 'real_images.png'))
    
    # Save sampled images
    fake_images = fake_images.reshape(fake_images.size(0), 1, 28, 28)
    save_image(denorm(fake_images), os.path.join(sample_dir, 'fake_images-{}.png'.format(epoch+1)))

RuntimeError: ignored

# 5 

In [0]:
import os 
import torch 
import torch.nn as nn 
import torch.nn.functional as F 
import torchvision 
from torchvision import transforms 
from torchvision.utils import save_image 

In [0]:
image_size = 784 
h_dim = 400 
z_dim = 20 
num_epochs = 15 
batch_size = 128 
learning_rate = 1e-3 

In [0]:
dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)

data_loader = torch.utils.data.DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True)

In [0]:
class VAE(nn.Module):
    def __init__(self, image_size=784, h_dim=400, z_dim=20):
        super(VAE, self).__init__()
        self.fc1 = nn.Linear(image_size, h_dim)
        self.fc2 = nn.Linear(h_dim, z_dim)
        self.fc3 = nn.Linear(h_dim, z_dim)
        self.fc4 = nn.Linear(z_dim, h_dim)
        self.fc5 = nn.Linear(h_dim, image_size)

    def encode(self, x):
        h = F.relu(self.fc1(x))
        return self.fc2(h), self.fc3(h)

    def reparameterize(self, mu, log_var):
        std = torch.exp(log_var/2)
        eps = torch.randn_like(std)
        return mu + eps * std 

    def decode(self, z):
        h = F.relu(self.fc4(z))
        return F.sigmoid(self.fc5(h))

    def forward(self, x):
        mu, log_var = self.encode(x)
        z = self.reparameterize(mu, log_var)
        x_reconst = self.decode(z)
        return x_reconst, mu, log_var

In [0]:
model = VAE()

In [0]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [0]:
for epoch in range(num_epochs):
    for i, (x, _) in enumerate(data_loader):
        x = x.view(-1, image_size)
        x_reconst, mu, log_var = model(x)

        reconst_loss = F.binary_cross_entropy(x_reconst, x, size_average=False)
        kl_div = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())

        loss = reconst_loss + kl_div 
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 10 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Reconst Loss: {:.4f}, KL Div: {:.4f}'
                .format(epoch+1, num_epochs, i+1, len(data_loader), reconst_loss.item(), kl_div.item()))

    with torch.no_grad():
        z = torch.randn(batch_size, z_dim)
        out = model.decode(z).view(-1, 1, 28, 28)
        save_image(out, os.path.join(sample_dir, 'sampled-{}.png'.format(epoch+1)))

        out, _, _ = model(x)
        x_concat = torch.cat([x.view(-1, 1, 28, 28), out.view(-1, 1, 28, 28)], dim=3)
        save_image(x_concat, os.path.join(sample_dir, 'reconst-{}.png'.format(epoch+1)))

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1350: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


Epoch [1/15], Step [10/469], Reconst Loss: 14143.3359, KL Div: 2709.1008
Epoch [1/15], Step [20/469], Reconst Loss: 13410.0645, KL Div: 2655.5310
Epoch [1/15], Step [30/469], Reconst Loss: 13167.0615, KL Div: 2742.2844
Epoch [1/15], Step [40/469], Reconst Loss: 13742.1484, KL Div: 2759.3984
Epoch [1/15], Step [50/469], Reconst Loss: 13679.2109, KL Div: 2753.9487
Epoch [1/15], Step [60/469], Reconst Loss: 13105.2725, KL Div: 2820.8018
Epoch [1/15], Step [70/469], Reconst Loss: 13143.0830, KL Div: 2680.2004
Epoch [1/15], Step [80/469], Reconst Loss: 14036.1504, KL Div: 2727.3857
Epoch [1/15], Step [90/469], Reconst Loss: 13381.2168, KL Div: 2876.7642
Epoch [1/15], Step [100/469], Reconst Loss: 13293.8623, KL Div: 2779.4324
Epoch [1/15], Step [110/469], Reconst Loss: 13604.4346, KL Div: 2755.4458
Epoch [1/15], Step [120/469], Reconst Loss: 12785.6367, KL Div: 2793.2695
Epoch [1/15], Step [130/469], Reconst Loss: 12676.8623, KL Div: 3005.2170
Epoch [1/15], Step [140/469], Reconst Loss: 129

# 6

In [0]:
!pip install argparse

In [0]:
!pip install Pillow

In [0]:
!ls

sample_data


In [0]:
from __future__ import division 
from torchvision import models 
from torchvision import transforms 
from PIL import Image 
import argparse 
import torch 
import torchvision 
import torch.nn as nn 
import numpy as np 

In [0]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [0]:
!mkdir png 

In [0]:
def load_image(image_path, transform=None, max_size=None, shape=None):
    image = Image.open(image_path)

    if max_size:
        scale = max_size / max(image.size)
        size = np.array(image.size) * scale 
        image = image.resize(size.astype(int), Image.ANTIALIAS)

    if shape:
        image = image.resize(shape, Image.LANCZOS)

    if transform:
        image = transform(image).unsqueeze(0)

    return image.to(device)

In [0]:
class VGGNet(nn.Module):
    def __init__(self):
        super(VGGNet, self).__init__()
        self.select = ['0', '5', '10', '19', '28']
        self.vgg = models.vgg19(pretrained=True).features 

    def forward(self, x):
        features = []
        for namr, layer in self.vgg_modules.items():
            x = layer(x)
            if name in self.select:
                features.append(x)
        return features

In [0]:
def main(config):
    transform = transforms.Compose([
        transforms.ToTensor(), 
        transforms.Normalize(mean=(0.485, 0.456, 0.406), 
                  std=(0.229, 0.224, 0.225))

    content = load_image(config.content, transform, max_size=config.max_size)
    style = load_image(config.style, transform, shape=[content.size(2), content.size(3)])

    target = content.clone().requires_grad_(True)
    optimizer = torch.optim.Adam([target], lr=config.lr, betas=[0.5, 0.999])
    vgg = VGGNet().to(device).eval()

    for step in range(config.total_step):
        target_features = vgg(target)
        content_features = vgg(content)
        style_features = vgg(style)

        style_loss = 0 
        content_loss = 0 
        for f1, f2, f3 in zip(target_features, content_features, style_features):
            content_loss += torch.mean((f1 - f2)**2)

            _, c, h, w = f1.size()
            f1 = f1.view(c, h * w)
            f3 = f3.view(c, h * w)

            f1 = torch.mm(f1, f1.t())
            f3 = torch.mm(f3, f3.t())

            style_loss += torch.mean((f1 - f3)**2) / (c * h * w)

        loss = content_loss + config.style_weight * style_loss 
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (step+1) % config.log_step == 0:
            print()

        if (step+1) % config.sample_step == 0:
            denorm = transforms.Normalize((-2.12, -2.04, -1.80), (4.37, 4.46, 4.44))
            img = target.clone().squeeze()
            img = denorm(img).clamp_(0, 1)
            torchvision.utils.save_image(img, 'output{}.png'.format(step+1))
    ])

# 7

In [0]:
!git clone https://github.com/yunjey/pytorch-tutorial.git

Cloning into 'pytorch-tutorial'...
remote: Enumerating objects: 816, done.
remote: Total 816 (delta 0), reused 0 (delta 0), pack-reused 816
Receiving objects: 100% (816/816), 12.78 MiB | 37.29 MiB/s, done.
Resolving deltas: 100% (432/432), done.


In [0]:
!ls

png  pytorch-tutorial  sample_data


In [0]:
!python ./pytorch-tutorial/tutorials/03-advanced/neural_style_transfer/main.py --content='./pytorch-tutorial/tutorials/03-advanced/neural_style_transfer/png/content.png'  --style='./pytorch-tutorial/tutorials/03-advanced/neural_style_transfer/png/style3.png'

Namespace(content='./pytorch-tutorial/tutorials/03-advanced/neural_style_transfer/png/content.png', log_step=10, lr=0.003, max_size=400, sample_step=500, style='./pytorch-tutorial/tutorials/03-advanced/neural_style_transfer/png/style.png', style_weight=100, total_step=2000)
Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/checkpoints/vgg19-dcbb9e9d.pth
100% 548M/548M [00:09<00:00, 57.5MB/s]
Step [10/2000], Content Loss: 6.8235, Style Loss: 113.8935
Step [20/2000], Content Loss: 10.2459, Style Loss: 85.9166
Step [30/2000], Content Loss: 12.3221, Style Loss: 71.9203
Step [40/2000], Content Loss: 13.6759, Style Loss: 62.9203
Step [50/2000], Content Loss: 14.6150, Style Loss: 56.5528
Step [60/2000], Content Loss: 15.3100, Style Loss: 51.6885
Step [70/2000], Content Loss: 15.8551, Style Loss: 47.7848
Step [80/2000], Content Loss: 16.2998, Style Loss: 44.5473
Step [90/2000], Content Loss: 16.6769, Style Loss: 41.8052
Step [100/2000], Content Loss: 1

In [0]:
!python ./pytorch-tutorial/tutorials/03-advanced/neural_style_transfer/main.py --content='./pytorch-tutorial/tutorials/03-advanced/neural_style_transfer/png/content.png'  --style='./pytorch-tutorial/tutorials/03-advanced/neural_style_transfer/png/style3.png'

Namespace(content='./pytorch-tutorial/tutorials/03-advanced/neural_style_transfer/png/content.png', log_step=10, lr=0.003, max_size=400, sample_step=500, style='./pytorch-tutorial/tutorials/03-advanced/neural_style_transfer/png/style3.png', style_weight=100, total_step=2000)
Step [10/2000], Content Loss: 2.6140, Style Loss: 146.3325
Step [20/2000], Content Loss: 5.1191, Style Loss: 109.2023
Traceback (most recent call last):
  File "./pytorch-tutorial/tutorials/03-advanced/neural_style_transfer/main.py", line 126, in <module>
    main(config)
  File "./pytorch-tutorial/tutorials/03-advanced/neural_style_transfer/main.py", line 99, in main
    loss.backward()
  File "/usr/local/lib/python3.6/dist-packages/torch/tensor.py", line 118, in backward
    torch.autograd.backward(self, gradient, retain_graph, create_graph)
  File "/usr/local/lib/python3.6/dist-packages/torch/autograd/__init__.py", line 93, in backward
    allow_unreachable=True)  # allow_unreachable flag
KeyboardInterrupt


# 8

In [0]:
import torch 

In [0]:
torch.__version__

'1.3.0+cu100'

In [0]:
%matplotlib inline 

In [0]:
from __future__ import print_function 
import torch 

In [0]:
x = torch.empty(5, 3)
print(x)

tensor([[4.6024e-36, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 2.8026e-45],
        [0.0000e+00, 1.1210e-44, 0.0000e+00],
        [1.4013e-45, 0.0000e+00, 0.0000e+00]])


In [0]:
x = torch.rand(5, 3)
print(x)

tensor([[0.4676, 0.1340, 0.6365],
        [0.7461, 0.1219, 0.6578],
        [0.8245, 0.4888, 0.5925],
        [0.4426, 0.8526, 0.2374],
        [0.8673, 0.0093, 0.2972]])


In [0]:
x = torch.zeros(5, 3, dtype=torch.long)
print(x)

tensor([[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]])


In [0]:
x = torch.tensor([5.5, 3])
print(x)

tensor([5.5000, 3.0000])


In [0]:
x = x.new_ones(5, 3, dtype=torch.double)
print(x)

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]], dtype=torch.float64)


In [0]:
x = torch.randn_like(x, dtype=torch.float)
print(x)

tensor([[-0.5479,  1.2288, -0.9941],
        [ 2.5916,  1.0550, -0.2982],
        [-0.8178, -1.8463,  0.1055],
        [ 0.9756, -0.1240,  1.4059],
        [-0.2842,  0.0840, -0.7878]])


In [0]:
print(x.size())

torch.Size([5, 3])


In [0]:
y = torch.rand(5, 3)
print(x + y)

tensor([[ 0.0490,  1.3681, -0.1916],
        [ 3.0397,  1.1281, -0.1447],
        [ 0.1274, -1.2173,  0.9539],
        [ 1.6774,  0.1409,  2.1909],
        [ 0.3377,  0.4043, -0.5149]])


In [0]:
print(torch.add(x, y))

tensor([[ 0.0490,  1.3681, -0.1916],
        [ 3.0397,  1.1281, -0.1447],
        [ 0.1274, -1.2173,  0.9539],
        [ 1.6774,  0.1409,  2.1909],
        [ 0.3377,  0.4043, -0.5149]])


In [0]:
result = torch.empty(5, 3)
torch.add(x, y, out=result)
print(result)

tensor([[ 0.0490,  1.3681, -0.1916],
        [ 3.0397,  1.1281, -0.1447],
        [ 0.1274, -1.2173,  0.9539],
        [ 1.6774,  0.1409,  2.1909],
        [ 0.3377,  0.4043, -0.5149]])


In [0]:
print(y)
y.add_(x)
print(y)

tensor([[ 0.0490,  1.3681, -0.1916],
        [ 3.0397,  1.1281, -0.1447],
        [ 0.1274, -1.2173,  0.9539],
        [ 1.6774,  0.1409,  2.1909],
        [ 0.3377,  0.4043, -0.5149]])
tensor([[-0.4989,  2.5969, -1.1858],
        [ 5.6313,  2.1832, -0.4428],
        [-0.6904, -3.0636,  1.0594],
        [ 2.6530,  0.0169,  3.5968],
        [ 0.0536,  0.4883, -1.3027]])


In [0]:
print(x)

tensor([[-0.5479,  1.2288, -0.9941],
        [ 2.5916,  1.0550, -0.2982],
        [-0.8178, -1.8463,  0.1055],
        [ 0.9756, -0.1240,  1.4059],
        [-0.2842,  0.0840, -0.7878]])


In [0]:
print(x[:, 1])

tensor([ 1.2288,  1.0550, -1.8463, -0.1240,  0.0840])


In [0]:
x = torch.randn(4, 4)
y = x.view(16)
z = x.view(-1, 8)
print(x.size(), y.size(), z.size())

torch.Size([4, 4]) torch.Size([16]) torch.Size([2, 8])


In [0]:
print(x)

tensor([[ 0.5960, -0.2749,  0.0981,  1.1148],
        [-2.3957,  0.4872,  1.1043,  1.4925],
        [-0.2685, -0.9455, -0.8301,  0.1530],
        [ 0.0690,  0.3921, -0.0575,  0.7194]])


In [0]:
print(x.item())

ValueError: ignored

In [0]:
x = torch.randn(1)
print(x)
print(x.item())

tensor([-1.0633])
-1.063294529914856


In [0]:
x =torch.ones(3, 4)
print(x)

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])


In [0]:
y = torch.zeros_like(x)
print(y)

tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])


In [0]:
a = torch.ones(5)
print(a)

tensor([1., 1., 1., 1., 1.])


In [0]:
b = a.numpy()
print(b)

[1. 1. 1. 1. 1.]


In [0]:
a.add_(1)
print(a)
print(b)

tensor([2., 2., 2., 2., 2.])
[2. 2. 2. 2. 2.]


In [0]:
c = torch.tensor(b)
print(c)

tensor([2., 2., 2., 2., 2.])


In [0]:
import numpy as np 

In [0]:
a = np.ones(5)

In [0]:
print(a)

[1. 1. 1. 1. 1.]


In [0]:
b = torch.from_numpy(a)
print(b)

tensor([1., 1., 1., 1., 1.], dtype=torch.float64)


In [0]:
np.add(a, 1, out=a)

array([2., 2., 2., 2., 2.])

In [0]:
print(a)

[2. 2. 2. 2. 2.]


In [0]:
print(b)

tensor([2., 2., 2., 2., 2.], dtype=torch.float64)


In [0]:
if torch.cuda.is_available():
    device = torch.device('cuda')
    y = torch.ones_like(x, device=device)
    x = x.to(device)
    z = x + y 
    print(z)
    print(z.to('cpu', torch.double))

# 9

In [0]:
import torch 

In [0]:
x = torch.ones(2, 2, requires_grad=True)
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


In [0]:
y = x + 2 
print(y)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


In [0]:
print(y.grad_fn)

In [0]:
z = y * y * 3 
print(z)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>)


In [0]:
out = z.mean()
print(out)

tensor(27., grad_fn=<MeanBackward0>)


In [0]:
a = torch.randn(2, 2)
print(a)

tensor([[ 0.0082,  0.2534],
        [-1.9240, -1.5676]])


In [0]:
a = ((a * 3) / (a - 1))
print(a.requires_grad)

False


In [0]:
a.requires_grad_(True)
print(a.requires_grad)

True


In [0]:
b = (a * a).sum()
print(b.grad_fn)

In [0]:
out.backward()

In [0]:
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


In [0]:
x = torch.randn(3, requires_grad=True)

y = x * 2 
while y.data.norm() < 1000:
    y = y * 2 

print(y)

tensor([1377.1154,  304.6030, 1172.8605], grad_fn=<MulBackward0>)


In [0]:
gradients = torch.tensor([0.1, 1.0, 0.0001], dtype=torch.float)
y.backward(gradients)

print(x.grad)

tensor([1.0240e+02, 1.0240e+03, 1.0240e-01])


In [0]:
print(x.requires_grad)

True


In [0]:
print((x ** 2).requires_grad)

True


In [0]:
with torch.no_grad():
    print((x ** 2).requires_grad)

False


# 10

In [0]:
%matplotlib inline 

In [0]:
import gym

In [0]:
import numpy as np 
np.random.seed(0)
import pandas as pd 

In [0]:
space_names = ['观测空间','动作空间','奖励范围','最大步数']
df = pd.DataFrame(columns=space_names)

In [0]:
env_specs = gym.envs.registry.all()

In [0]:
for env_spec in env_specs:
    env_id = env_spec.id
    env = gym.make(env_id)
    observation_space = env.observation_space 
    action_space = env.action_space 
    reward_range = env.reward_range 
    max_episode_steps = None 
    if isinstance(env, gym.wrappers.time_limit.TimeLimit):
        max_episode_steps = env._max_episode_steps 
    df.loc[env_id] = [observation_space, action_space, reward_range, max_episode_steps]

with pd.option_context('display.max_rows', None):
    display(df)

ModuleNotFoundError: ignored

In [0]:
space_names = ['观测空间', '动作空间', '奖励范围', '最大步数']
df = pd.DataFrame(columns=space_names)

env_specs = gym.envs.registry.all()
for env_spec in env_specs:
    env_id = env_spec.id
    try:
        env = gym.make(env_id)
        observation_space = env.observation_space
        action_space = env.action_space
        reward_range = env.reward_range
        max_episode_steps = None
        if isinstance(env, gym.wrappers.time_limit.TimeLimit):
            max_episode_steps = env._max_episode_steps
        df.loc[env_id] = [observation_space, action_space, reward_range, max_episode_steps]
    except:
        pass

with pd.option_context('display.max_rows', None):
    display(df)

,观测空间,动作空间,奖励范围,最大步数
Copy-v0,Discrete(6),"Tuple(Discrete(2), Discrete(2), Discrete(5))","(-inf, inf)",200
RepeatCopy-v0,Discrete(6),"Tuple(Discrete(2), Discrete(2), Discrete(5))","(-inf, inf)",200
ReversedAddition-v0,Discrete(4),"Tuple(Discrete(4), Discrete(2), Discrete(3))","(-inf, inf)",200
ReversedAddition3-v0,Discrete(4),"Tuple(Discrete(4), Discrete(2), Discrete(3))","(-inf, inf)",200
DuplicatedInput-v0,Discrete(6),"Tuple(Discrete(2), Discrete(2), Discrete(5))","(-inf, inf)",200
Reverse-v0,Discrete(3),"Tuple(Discrete(2), Discrete(2), Discrete(2))","(-inf, inf)",200
CartPole-v0,"Box(4,)",Discrete(2),"(-inf, inf)",200
CartPole-v1,"Box(4,)",Discrete(2),"(-inf, inf)",500
MountainCar-v0,"Box(2,)",Discrete(3),"(-inf, inf)",200
MountainCarContinuous-v0,"Box(2,)","Box(1,)","(-inf, inf)",999


In [0]:
env = gym.make('MountainCar-v0')

In [0]:
print('observation = {}'.format(env.observation_space))

observation = Box(2,)


In [0]:
print('action = {}'.format(env.action_space))

action = Discrete(3)


In [0]:
print('range = {} - {}'.format(env.observation_space.low, env.observation_space.high))

range = [-1.2  -0.07] - [0.6  0.07]


In [0]:
print('action num = {}'.format(env.action_space.n))

action num = 3


In [0]:
class BespokeAgent:
    def __init__(self, env):
        pass 

    def decide(self, observation):
        position, velocity = observation 
        lb = min(-0.09 * (position + 0.25) ** 2 + 0.03, 0.3 * (position + 0.9) ** 4 - 0.008)
        ub = -0.07 * (position + 0.38) ** 2 + 0.07 
        if lb < velocity < ub:
            action = 2 
        else:
            action = 0 
        return action 

    def learn(self, *args):
        pass 
            

In [0]:
agent = BespokeAgent(env)

In [0]:
def play_montecarlo(env, agent, render=False, train=False):
    episode_reward = 0. 
    observation = env.reset()
    while True:
        if render:
            env.render()
        action = agent.decide(observation)
        next_observation, reward, done, _ = env.step(action)
        episode_reward += reward 
        if train:
            agent.learn(observation, action, reward, done)
        if done:
            break 
        observation = next_observation 
    return episode_reward

In [0]:
env.seed(0)

[0]

In [0]:
episode_reward = play_montecarlo(env, agent, render=True)

NoSuchDisplayException: ignored

In [0]:
def play_montecarlo(env, agent, render=False, train=False):
    episode_reward = 0. # 记录回合总奖励，初始化为0
    observation = env.reset() # 重置游戏环境，开始新回合
    while True: # 不断循环，直到回合结束
        if render: # 判断是否显示
            env.render() # 显示图形界面，图形界面可以用 env.close() 语句关闭
        action = agent.decide(observation)
        next_observation, reward, done, _ = env.step(action) # 执行动作
        episode_reward += reward # 收集回合奖励
        if train: # 判断是否训练智能体
            agent.learn(observation, action, reward, done) # 学习
        if done: # 回合结束，跳出循环
            break
        observation = next_observation
    return episode_reward # 返回回合总奖励

In [0]:
env.seed(0) # 设置随机数种子,只是为了让结果可以精确复现,一般情况下可删去
episode_reward = play_montecarlo(env, agent, render=True)
print('回合奖励 = {}'.format(episode_reward))
env.close() # 此语句可关闭图形界面